In [ ]:
# The purpose of this notebook is to find out what's the smallest network 
# that can be used to get max prediction score

In [ ]:
from sklearn import metrics

In [ ]:
# Neural Nets imports
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta, Adagrad, Adamax, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [ ]:
verboseLevel=0
validationSplit=0.2
batchSize=30
epochs=1000

In [ ]:
# callback preparation
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.5,
                              patience=2,
                              verbose=verboseLevel,
                              mode='min',
                              min_lr=0.001)

In [ ]:
from tensorflow.keras.utils import multi_gpu_model

In [ ]:
inputSize = 9
colList = ['HiddenLayers', 'R2Score', 'MAE', 'MSE', 'RMSE', 'H5FileName', 'TrainHistory', 'TrainPredictions']

In [ ]:
# Check Large Dataset first

In [ ]:
def createModel(dataFrame, layerSize, loops, y_train, X_train, y_test, X_test, targetScaler, labelSet):
    #print(f'Creating model using layer size = {layerSize} on set = {labelSet}.\n')
    i = loops
    #print(f'Training on {i} hidden layers\n')
    model = Sequential()
    model.add(Dense(layerSize, kernel_initializer='normal',
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                    input_dim=inputSize, activation=custom_activation))
    for j in range(i):
        model.add(Dense(layerSize, 
                        #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                        kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                        #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                        kernel_initializer='normal', activation=custom_activation))
    model.add(BatchNormalization())
    model.add(Dense(1, kernel_initializer='normal', 
                    #kernel_regularizer=l1(0.01), bias_regularizer=l1(0.01),
                    kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01),
                    #kernel_regularizer=l1_l2(0.01), bias_regularizer=l1_l2(0.01),
                    activation='linear'))

    # only set this if GPU capabilities available
    # model = multi_gpu_model(model, gpus=2)

    optmzr=Adam(lr=0.001)    
    model.compile(optimizer=optmzr, loss='mae', metrics=['mae'])

    model_h5_name = 'mlp_' + str(layerSize)+ '_' + str(i) + '_model_minMax_' + labelSet + '_L2.h5'
    checkpoint_nn_minMax = ModelCheckpoint(model_h5_name,
                             monitor='val_loss',
                             verbose=verboseLevel,
                             save_best_only=True,
                             mode='min')
    callbacks_list_nn_minMax = [checkpoint_nn_minMax, reduce_lr]

    history_MLP_minMax = model.fit(X_train.to_numpy(), y_train,
                                batch_size=batchSize, 
                                validation_split=validationSplit, 
                                epochs=epochs, verbose=verboseLevel,
                                callbacks=callbacks_list_nn_minMax)

    #reload the best model!
    model_new = load_model(model_h5_name)
    #Predict
    y_pred_scaled = model_new.predict(X_test.to_numpy())
    #Evaluate metrics
    y_pred = targetScaler.inverse_transform(y_pred_scaled)
    r2_score = metrics.r2_score(y_test, y_pred)
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    #store values
    row = [i, r2_score, mae, mse, rmse, model_h5_name, history_MLP_minMax, y_pred]
    df = pd.DataFrame(np.array(row).reshape(1, len(colList)), columns=colList)
    dataFrame = dataFrame.append(df, ignore_index=True)

    tf.keras.backend.clear_session()
    del(model)
    del(model_new)
        
    return dataFrame


In [ ]:
#### Base MinMax

In [ ]:
%%time
dataFrame128_1 = pd.DataFrame(columns=colList)
layerSize = 128
loops = 3
dataFrame128_1 = createModel(dataFrame128_1, layerSize, loops, 
                        y_train_scaled_minMax_base, X_train_base_minMax,
                        y_test1_summaryL1Z2_minMax, X_test1_summaryL1Z2_minMax, 
                        targetMinMaxScalerBase, 'base')


In [ ]:
# Determine the IDX value where the MAE is smallest
minMaeIDX = dataFrame128_1.loc[dataFrame128_1['MAE']==dataFrame128_1['MAE'].min()].index[0]

In [ ]:
dataFrame128_1.iloc[minMaeIDX]

In [ ]:
#import joblib

In [ ]:
#save the scaler
#joblib.dump(targetMinMaxScalerBase, 'targetTrainMinMaxScalerBase.pkl')
#joblib.dump(minMaxScalerBase, 'trainMinMaxScalerBase.pkl')

In [ ]:
#if dataFrame128_1.iloc[minMaeIDX]['MAE'] < minMaeAWS_base :
minMseAWS_base = dataFrame128_1.iloc[minMaeIDX]['MSE']
minRmseAWS_base = dataFrame128_1.iloc[minMaeIDX]['RMSE']
minMaeAWS_base = dataFrame128_1.iloc[minMaeIDX]['MAE']
minR2AWS_base = dataFrame128_1.iloc[minMaeIDX]['R2Score']
modelNameAWS_base = "MLP_128_Base_MinMax"
posAWS_base = minMaeIDX

In [ ]:
y_pred_MLP_MinMax = dataFrame128_1['TrainPredictions'][minMaeIDX]

In [ ]:
delta = pd.DataFrame()
delta = (1-(pd.DataFrame(y_pred_MLP_MinMax)  / y_test1_summaryL1Z2_minMax.to_numpy()))*100
delta = delta.rename(columns={delta.columns[0]: "ThroughputDeltaDeviationPercentage"})

In [ ]:
mpd_base = delta.median()[0]

In [ ]:
def mape(test, pred):
    x = 0
    for i in range(len(test)):
        x+=np.abs((test[test.columns[0]][i] - pred[i])/test[test.columns[0]][i])
    return (x/len(test)*100)

In [ ]:
mape_base = mape(y_test1_summaryL1Z2_minMax, y_pred_MLP_MinMax)

In [ ]:
minMaeAWS_base, minR2AWS_base, modelNameAWS_base, posAWS_base, minMseAWS_base, minRmseAWS_base, mape_base, mpd_base